In [123]:
import pandas as pd
import numpy as np
import gc
from dateutil.parser import parse  # 解析日期

import os
import matplotlib.pyplot as plt

In [124]:
from tqdm import tqdm
from multiprocessing import Pool

In [126]:
cd ../00_data/

/nas/00_data


# utils 

In [127]:
# 每一种数据按对应的数据类型，压缩空间
# 对各种数据类型分析很清楚
def creatDtype():
    dtype = {'id':'object',
             'label':'int8',
             'date':'int64',
             'f1':'uint8',
             'f2': 'uint8',
             'f3': 'uint8',
             'f4': 'uint8',
             'f5': 'float32',
             'ndays':'uint8'
             }
    for i in range(20,298):
        dtype['f'+str(i)] = 'float32'
    for i in range(6,20):
        dtype['f'+str(i)] = 'uint8'
    return dtype

In [83]:
def get_col_null_rate(df, thres = 0.3):
    # 计算输入的df中各col null的比例
    # input  -- df of data
    # output -- df index and null rate
    info = pd.DataFrame()
    info['isnull'] = df.isnull().sum()
    info['null_ratio'] = info['isnull'] / df.shape[0]
    info = info.sort_values(by = 'null_ratio', ascending=False)
    info = info[info['null_ratio'] > thres]

    return info

# 数据读取

In [7]:
train_data = pd.read_csv('atec_anti_fraud_train.csv',dtype=creatDtype())

In [8]:
test_data = pd.read_csv('atec_anti_fraud_test_b.csv',dtype=creatDtype())

In [147]:
print(sorted(train_data['date'].unique()))

[20170905, 20170906, 20170907, 20170908, 20170909, 20170910, 20170911, 20170912, 20170913, 20170914, 20170915, 20170916, 20170917, 20170918, 20170919, 20170920, 20170921, 20170922, 20170923, 20170924, 20170925, 20170926, 20170927, 20170928, 20170929, 20170930, 20171001, 20171002, 20171003, 20171004, 20171005, 20171006, 20171007, 20171008, 20171009, 20171010, 20171011, 20171012, 20171013, 20171014, 20171015, 20171016, 20171017, 20171018, 20171019, 20171020, 20171021, 20171022, 20171023, 20171024, 20171025, 20171026, 20171027, 20171028, 20171029, 20171030, 20171031, 20171101, 20171102, 20171103, 20171104, 20171105]


In [148]:
print(sorted(test_data['date'].unique()))

[20180206, 20180207, 20180208, 20180209, 20180210, 20180211, 20180212, 20180213, 20180214, 20180215, 20180216, 20180217, 20180218, 20180219, 20180220, 20180221, 20180222, 20180223, 20180224, 20180225, 20180226, 20180227, 20180228, 20180301, 20180302, 20180303, 20180304, 20180305, 20180306]


In [128]:
print('unique date num of train set: {}'.format(len(train_data['date'].unique())))
print('unique date num of test set: {}'.format(len(test_data['date'].unique())))

unique date num of train set: 62
unique date num of test set: 29


In [129]:
train_data['ndays'] = train_data['date'].apply(lambda x:(parse(str(x))-parse(str(20170905))).days)

In [130]:
test_data['ndays'] = test_data['date'].apply(lambda x:(parse(str(x))-parse(str(20170905))).days)

# 特征的缺失率分析

分析可以看出：

- 测试集中高缺失率的特征比训练集中更多
- 测试集高缺失率的特征集合包含训练集中高缺失率集合，总共127

In [133]:
# 分别统计训练和测试集的特征缺失率，阈值0.3
train_null_cols = get_col_null_rate(train_data, thres=0.3)
train_null_cols = list(train_null_cols.index)
test_null_cols = get_col_null_rate(test_data, thres=0.3)
test_null_cols = list(test_null_cols.index)

In [149]:
train_null_cols2 = get_col_null_rate(train_data, thres=0.3)

In [153]:
# train_null_cols2

In [134]:
print('train set null rate>0.3 cols: {}'.format(len(train_null_cols)))
print('test set null rate>0.3 cols: {}'.format(len(test_null_cols)))

train set null rate>0.3 cols: 30
test set null rate>0.3 cols: 127


In [136]:
null_all_cols = set(test_null_cols + test_null_cols)
print('train + test set null rate>0.3 cols: {}'.format(len(null_all_cols)))

train + test set null rate>0.3 cols: 127


In [138]:
print(sorted(null_all_cols))

['f100', 'f101', 'f102', 'f103', 'f104', 'f105', 'f106', 'f107', 'f108', 'f109', 'f110', 'f111', 'f112', 'f113', 'f114', 'f115', 'f116', 'f117', 'f118', 'f119', 'f120', 'f121', 'f122', 'f123', 'f124', 'f125', 'f126', 'f127', 'f128', 'f129', 'f130', 'f131', 'f132', 'f133', 'f134', 'f135', 'f136', 'f137', 'f138', 'f139', 'f140', 'f141', 'f142', 'f143', 'f144', 'f145', 'f146', 'f147', 'f148', 'f149', 'f150', 'f151', 'f152', 'f153', 'f154', 'f155', 'f156', 'f157', 'f158', 'f159', 'f160', 'f32', 'f33', 'f34', 'f35', 'f36', 'f37', 'f38', 'f39', 'f40', 'f41', 'f42', 'f43', 'f44', 'f45', 'f46', 'f47', 'f48', 'f49', 'f50', 'f51', 'f54', 'f55', 'f56', 'f57', 'f58', 'f59', 'f60', 'f61', 'f62', 'f63', 'f64', 'f65', 'f66', 'f67', 'f68', 'f69', 'f70', 'f71', 'f72', 'f73', 'f74', 'f75', 'f76', 'f77', 'f78', 'f79', 'f80', 'f81', 'f82', 'f83', 'f84', 'f85', 'f86', 'f87', 'f88', 'f89', 'f90', 'f91', 'f92', 'f93', 'f94', 'f95', 'f96', 'f97', 'f98', 'f99']


# EDA

In [28]:
data = train_data.append(test_data)

In [139]:
print('train+test data set shape: {}'.format(data.shape))

train+test data set shape: (1495269, 301)


## 从缺失率的角度分析数据的稳定性

In [140]:
def draw_feature_debug(p):
    # p = 'f24'
    # 计算一个fe 在每天的数据缺失率
    miss_rate = pd.DataFrame(data.groupby('ndays')[p].apply(lambda x: sum(pd.isnull(x))) / data.groupby('ndays')['ndays'].count()).reset_index()
    miss_rate.columns = ['ndays', p]
    # 一个fe在每天的分组
    value_dist = pd.DataFrame(data.groupby('ndays')[p].mean()).reset_index()

    plt.figure(figsize=(10,4),dpi=98)
    p1 = plt.subplot(121)
    p2 = plt.subplot(122)

    # 特征缺失率的变化
    p1.plot(miss_rate['ndays'], miss_rate[p])
    p1.axvline(61, color='r')
    p1.axvline(153, color='r')
    p1.axhline(0.5, color='y')
    p1.axis([0, 200, 0, 1])
    p1.set_xlabel('ndays')
    p1.set_ylabel('miss_rate_' + p)
    p1.set_title('miss_rate_' + p)

    # 特征数值的变化
    p2.plot(value_dist['ndays'], value_dist[p])
    p2.axvline(61, color='r')
    p2.axvline(153, color='r')
    p2.axhline(1, color='y')
    p2.set_xlabel('ndays')
    p2.set_ylabel('mean_of_' + p)
    p2.set_title('distribution of ' + p)
    plt.savefig('./debug/{}.jpg'.format(p))
#     plt.show()
    plt.close()    

In [141]:
# draw_feature_debug('f24')

In [102]:
fe_all = [fe for fe in data.columns if 'f' in fe]

In [102]:
with Pool(32) as p:
    list(tqdm(p.imap(draw_feature_debug, fe_all), total=len(fe_all)))

In [118]:
# 凤凰电冰箱文章指定删除的特征
need_del = ['f64','f65','f66','f67','f68','f69','f70','f71','f111','f112','f113','f114',
            'f115','f116','f117','f118','f119','f120','f121','f122','f123','f124','f125',
            'f126','f127','f128','f129','f130','f131','f132','f133','f134','f135','f136',
            'f137','f138','f139','f140','f141','f142','f143','f144','f145','f146','f147',
            'f148','f149','f150','f151','f152','f153','f154','f161','f162','f163','f164',
            'f165','f211','f212','f213','f214','f215','f216','f217','f218','f219','f220',
            'f221','f222','f223','f224','f225','f226','f227','f228','f229','f230','f231','f232','f233']
for i in range(20,54):
    need_del.append('f'+str(i))

In [164]:
fe_del = list(null_all_cols)

fe_del += ['f' + str(fe) for fe in range(20,28)]
fe_del += ['f' + str(fe) for fe in range(161,165)]
fe_del += ['f' + str(fe) for fe in range(211,234)]

In [171]:
print('fes to remove {} '.format(len(fe_del)))

fes to remove 162 
